In [26]:
import pandas
from IPython.display import display

def read_data(file_name):
    csv_file = pandas.read_csv(f'../data/{file_name}.csv')
    return csv_file

student_course_identifier = ["code_module", "code_presentation", "id_student"]

student_info = read_data('studentInfo')
student_info.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


In [27]:
# merge studentinfo 
# with studentRegistration
student_registration = read_data("studentRegistration")
students_merged_step_1 = pandas.merge(student_info, student_registration, on=student_course_identifier)
students_merged_step_1.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,NaN
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,NaN
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,-92.0,12.0
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-52.0,NaN
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-176.0,NaN


In [28]:
# merge studentinfo and studentRegistration
# with courses
courses = read_data("courses")
students_merged_step_2 = pandas.merge(students_merged_step_1, courses, on=['code_module', 'code_presentation'])
students_merged_step_2.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration,module_presentation_length
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,NaN,268
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,NaN,268
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,-92.0,12.0,268
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-52.0,NaN,268
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-176.0,NaN,268


In [29]:
# prepare vles for merging later
vles = pandas.merge(read_data("vle"), read_data("studentVle"), on=['code_module', 'code_presentation', 'id_site'])
vles.head()

,id_site,code_module,code_presentation,activity_type,week_from,week_to,id_student,date,sum_click
0,546943,AAA,2013J,resource,NaN,NaN,75091,-10,1
1,546943,AAA,2013J,resource,NaN,NaN,186149,-10,1
2,546943,AAA,2013J,resource,NaN,NaN,205350,-10,2
3,546943,AAA,2013J,resource,NaN,NaN,1626710,-9,1
4,546943,AAA,2013J,resource,NaN,NaN,2643002,-8,1


In [30]:
# group vle clicks per day
grouped_vles_per_day = vles.groupby(["code_module", "code_presentation", "id_student", "id_site", "date", "activity_type"]).agg({
    "sum_click": "sum"
}).reset_index()

In [31]:
# combine vle data as a single column value
grouped_vles_per_day["vles"] = grouped_vles_per_day[["id_site", "date", "activity_type", "sum_click"]].values.tolist()     
grouped_vles_per_day.head()

,code_module,code_presentation,id_student,id_site,date,activity_type,sum_click,vles
0,AAA,2013J,11391,546614,-5,homepage,7,"[546614, -5, homepage, 7]"
1,AAA,2013J,11391,546614,0,homepage,10,"[546614, 0, homepage, 10]"
2,AAA,2013J,11391,546614,1,homepage,9,"[546614, 1, homepage, 9]"
3,AAA,2013J,11391,546614,2,homepage,3,"[546614, 2, homepage, 3]"
4,AAA,2013J,11391,546614,6,homepage,1,"[546614, 6, homepage, 1]"


In [32]:
# combine all seperate from rows to a single row with a list
grouped_vles_per_student = grouped_vles_per_day.groupby(student_course_identifier, as_index=False).agg({
    "vles": lambda x: list(x)
})
grouped_vles_per_student.head()

,code_module,code_presentation,id_student,vles
0,AAA,2013J,11391,"[[546614, -5, homepage, 7], [546614, 0, homepa..."
1,AAA,2013J,28400,"[[546614, -10, homepage, 11], [546614, -8, hom..."
2,AAA,2013J,30268,"[[546614, -10, homepage, 3], [546614, -9, home..."
3,AAA,2013J,31604,"[[546614, -10, homepage, 11], [546614, -9, hom..."
4,AAA,2013J,32885,"[[546614, -10, homepage, 13], [546614, -9, hom..."


In [34]:
# merge studentinfo, studentRegistration and courses
# with vles and studentvles
students_merged_step_3 = pandas.merge(grouped_vles_per_student, students_merged_step_2, on=student_course_identifier)
students_merged_step_3.loc[0]

code_module                                                                 AAA
code_presentation                                                         2013J
id_student                                                                11391
vles                          [[546614, -5, homepage, 7], [546614, 0, homepa...
gender                                                                        M
region                                                      East Anglian Region
highest_education                                              HE Qualification
imd_band                                                                90-100%
age_band                                                                   55<=
num_of_prev_attempts                                                          0
studied_credits                                                             240
disability                                                                    N
final_result                            